## **Combining Data for use in Tableau**

We'd like to create an interactive dashboard for tableau, but to do that, we'll need to combine our data and add some new columns. Some features we'll need to add to the data include:
* `date_ord` - an ordinal column for the date that can be used to align states based on their closing/reopening announcement date. Thinking of using a +/- shutdown date to create this column.
* `is_reopen` - a boolean column that will say whether the row is from the state's opening or closing phase. Will do this by applying a True/False to all reopening or closing dataframes before doing a union.

In [261]:
import os
import csv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone

In [240]:
d1 = datetime(2020, 5, 23)

In [241]:
d2 = datetime(2020, 5, 26)

In [242]:
(d1 - d2).total_seconds() / 60

-4320.0

In [281]:
test = pd.read_csv('./data/sentiment_data/GA_reopen_sentiment.csv')

In [282]:
timeline_dict = {
    'FL': ['2020-04-01', '2020-04-29'],
    'TX': ['2020-03-31', '2020-05-18'],
    'NJ': ['2020-03-16', '2020-06-09'],
    'NY': ['2020-03-22', '2020-06-13'],
    'IL': ['2020-03-20', '2020-05-29'],
    'GA': ['2020-04-02', '2020-06-01'],
    'OH': ['2020-03-22', '2020-05-26'],
    'MI': ['2020-03-23', '2020-06-08']
}

In [283]:
# converting date to datetime
test['date'] = pd.to_datetime(test['date'])

# setting date as index
test.set_index('date', inplace=True, drop=False)

# converting date to ordinal
test['date_announcement_normalized'] = test['date'].apply(lambda x: x.toordinal())

In [284]:
test['date_announcement_normalized']['2020-06-02']

date
2020-06-02 00:00:18-04:00    737578
2020-06-02 00:00:39-04:00    737578
2020-06-02 00:01:23-04:00    737578
2020-06-02 00:01:28-04:00    737578
2020-06-02 00:02:34-04:00    737578
                              ...  
2020-06-02 23:58:48-04:00    737578
2020-06-02 23:59:19-04:00    737578
2020-06-02 23:59:40-04:00    737578
2020-06-02 23:59:49-04:00    737578
2020-06-02 23:59:53-04:00    737578
Name: date_announcement_normalized, Length: 6174, dtype: int64

In [285]:
test['hour']= test['date'].dt.strftime('%H')

In [286]:
# Announcement Date Ordinal
announcement_day = test[f'{timeline_dict["GA"][1]} 12']['date'][0]

In [287]:
day_2 = test['2020-06-02 10']['date'][0]

In [291]:
test['date_announcement_normalized'] = test.apply(lambda x: (x['date'] - announcement_day).total_seconds() // 60 ** 2, axis=1)

In [280]:
announcement_day_ord

737577

In [237]:
announcement_day_hour_ord

'73757712'

In [292]:
test['date_announcement_normalized']

date
2020-05-28 20:00:04-04:00    -89.0
2020-05-28 20:00:07-04:00    -89.0
2020-05-28 20:00:16-04:00    -89.0
2020-05-28 20:00:34-04:00    -88.0
2020-05-28 20:00:37-04:00    -88.0
                             ...  
2020-06-10 19:57:53-04:00    223.0
2020-06-10 19:57:54-04:00    223.0
2020-06-10 19:58:09-04:00    223.0
2020-06-10 19:59:11-04:00    223.0
2020-06-10 19:59:24-04:00    223.0
Name: date_announcement_normalized, Length: 86435, dtype: float64

In [173]:
if 'GA_shutdown_sentiment'.split('_')[1] == 'shutdown':
    test['is_reopen'] = False
else:
    df['is_reopen'] = True

In [62]:
test['is_reopen']

date
2020-05-28 20:00:04-04:00    True
2020-05-28 20:00:07-04:00    True
2020-05-28 20:00:16-04:00    True
2020-05-28 20:00:34-04:00    True
2020-05-28 20:00:37-04:00    True
                             ... 
2020-06-10 19:57:53-04:00    True
2020-06-10 19:57:54-04:00    True
2020-06-10 19:58:09-04:00    True
2020-06-10 19:59:11-04:00    True
2020-06-10 19:59:24-04:00    True
Name: is_reopen, Length: 86435, dtype: bool

In [304]:
def combine_and_add_cols(rel_path='./data/sentiment_data'):
    '''
    This function combines and adds columns to the csvs that
    contain sentiment analysis data. Specifically, this function
    adds the following features:
    
    date_ord: creates an ordinal value from the date column
    
    -----------------------
    Parameters:
    
    rel_path : str, the path where all of your csvs are stored
    '''    
    # Getting all of a state's filenames
    filenames = [filename for filename in os.listdir(rel_path) if filename.endswith('.csv')]
    
    # Create new df so the final df doesn't keep appending if you need to run again
    # First, we'll need to get the headers - thanks Tyler on SO for this
    # https://stackoverflow.com/questions/24962908/how-can-i-read-only-the-header-column-of-a-csv-file-using-python
#     with open(f'{rel_path}/{filenames[0]}', 'r') as f:
#         reader = csv.DictReader(f)
#         fieldnames = reader.fieldnames

#     # Write a clean df
#     pd.DataFrame(columns=fieldnames + ['']).to_csv(f'{rel_path}/combined/all_states_and_dates_sentiment.csv', index=False) # THIS DIDNT" NEED TO BE A VARIABLE ++++ REMOVE WHEN DONE
    
    # We'll need this dict for later, the values are organized as [shutdown_date_announced, reopening_date]
    timeline_dict = {
        'FL': ['2020-04-01', '2020-04-29'],
        'TX': ['2020-03-31', '2020-05-18'],
        'NJ': ['2020-03-16', '2020-06-09'],
        'NY': ['2020-03-22', '2020-06-13'],
        'IL': ['2020-03-20', '2020-05-29'],
        'GA': ['2020-04-02', '2020-06-01'],
        'OH': ['2020-03-22', '2020-05-26'],
        'MI': ['2020-03-23', '2020-06-08']
    }
        
    # Main combination loop
    for filename in filenames:
        df = pd.read_csv(f'{rel_path}/{filename}')
        
        #-----------IS REOPEN-------------
        '''
        We want to add a column that tells us if the row is 
        from reopening or shutdown data, we'll do this by
        referencing the filenames since that data is included there
        '''
        
        if filename.split('_')[1] == 'shutdown':
            df['is_reopen'] = False
            is_reopen = False # Setting this for later use
        else:
            df['is_reopen'] = True
            is_reopen = True # Setting this for later use
            
        
        #--------ORDINAL NORMALIZATION-----------
        '''
        The following code will set the shutdown/reopening 
        date as 0, so we can align states on a graph based on this date.
        '''
        
        # converting date to datetime
        df['date'] = pd.to_datetime(df['date'])

        # setting date as index
        df.set_index('date', inplace=True, drop=False)
        
        # State name pulled from filename
        state_name = filename.split('_')[0]
        
        announce_date = timeline_dict[state_name][is_reopen]
        
        # Setting the announcement day and hour (since not every dataset has noon, 
        # we're using the last tweet from the announcement day as our midpoint
        announcement_day_hour = df[f'{announce_date}']['date'][len(df[announce_date])//2] # this will pick the middle entry from the announcement day

        # Take the time delta in seconds divided by 60 ** 2 to get nubmer of hours
        df['date_announcement_normalized'] = df.apply(lambda x: (x['date'] - announcement_day_hour).total_seconds() // 60 ** 2, axis=1)

            
        #----------WRITE TO FINAL DF--------
        
        if filename == filenames[0]:
            # Write to final df
            df.to_csv(f'{rel_path}/combined/all_states_and_dates_sentiment.csv', index=False)
        else:
            # Append to final df
            df.to_csv(f'{rel_path}/combined/all_states_and_dates_sentiment.csv', index=False, mode='a', header=False)

In [305]:
combine_and_add_cols('./data/sentiment_data')